In [10]:
from __future__ import print_function
import cv2
import numpy as np
import os
from xml.dom import minidom
import matplotlib.path as mplPath
import numpy as np
import openslide
import cv2
import time
# import argparse

from multiprocessing import Process
import threading
import glob


In [11]:
# padd = 112
level = 0
openslide_formats = ['.ndpi', '.svs', '.tif', '.vms', '.vmu', '.scn', '.mrxs', '.tiff', 'svslide', 'bif']
#parser=argparse.ArgumentParser()
#parser.add_argument('-c','--case',help='cases')
# results=parser.parse_args()
# cases=results.case

def get_all_regions(file_name):
    mydoc = minidom.parse(file_name)
    annotations = mydoc.getElementsByTagName('Annotation')
    all_anns = []
    all_orgs = []
    all_paths = []
    name=[]
    for annotation in annotations:
        regions = annotation.getElementsByTagName('Region')
        name.append(annotation.getAttribute('Name').encode('utf-8'))
        all_regions = []
        orgs = []
        paths = []
        for region in regions:
            verticies = region.getElementsByTagName('Vertex')
            xy = []
            xy_path = []
            for item in verticies:
            	# print(item.getAttribute('X'))
            	# break
                xy.append(list(map(int,[float(item.getAttribute('X').encode('utf-8')),float(item.getAttribute('Y').encode('utf-8'))])))
                xy_path.append([item.getAttribute('X'),item.getAttribute('Y')])
            all_regions.append(xy)
            ox,oy,wd,ht = cv2.boundingRect(np.asarray(xy))
            orgs.append([ox,oy,wd,ht])
            paths.append(mplPath.Path(xy_path))
        all_anns.append(all_regions)
        all_orgs.append(orgs)
        all_paths.append(paths)
    return all_anns,all_orgs,all_paths,name




def get_annotations_1(format_f,xml_file_path,output_directory,padd,image_path,level=0):
    '''
    format_f = format of file
    padd = padding
    directory = ?
    file_name = name of the xml file
    path = ?
    lrp1 = ?
    
    '''
    ffl = len(format_f)
    ff = format_f
    padd = padd
    directory = output_directory #+ '/'
    all_anns,all_origins,all_paths,name = get_all_regions(xml_file_path)
    if(str(ff) in openslide_formats):
        img = openslide.open_slide(image_path)
    else:
        img = cv2.imread(image_path)

    count = 0
    pathsplt = image_path.split('/')
    #lwp1 = len(pathsplt)
    # subfold_count = lrp1 - lwp1
 #    #print(pathsplt)
 #    #print(subfold_count)
    # while(subfold_count < -1):
    #   if not os.path.exists(directory + pathsplt[subfold_count]):
    #       os.mkdir(directory + pathsplt[subfold_count])
    #   directory = directory + pathsplt[subfold_count] + '/'
 #        subfold_count = subfold_count + 1

    for lyr in range(len(all_anns)):
        all_regions = all_anns[lyr]
      #  if (name[lyr].decode("utf-8")!="IDC"):
      #      print("NO IDC")
      #      continue
        if not os.path.exists(output_directory + str(lyr)):
            os.makedirs(output_directory + str(lyr))
        if(len(all_regions) == 0):
            continue
        dir_path = directory + str(lyr) + '/'
        origins = all_origins[lyr]
        paths = all_paths[lyr]
        n_regions = len(all_regions)
        for region_id in range(n_regions):
            if(len(origins[region_id])==0):
                print("Y")
                continue
            ox,oy,w,h = origins[region_id]
            print("ox and oy: ",w,h)
            print("W and H: ",w,h)
            # print(type(w))
#             if (w>=40000 or h>=40000):
#                 continue
#             else:
            region = all_regions[region_id]
            # mpl_path = paths[tmp1]
            start_x = max(0 , ox - padd)
            start_y = max(0 , oy - padd)
            shifted_region = [[item[0] - start_x, item[1] - start_y] for item in region]
            mask_im = np.zeros((h + 2*padd, w + 2*padd, 3))
            mask_im = cv2.drawContours(mask_im, np.asarray([shifted_region]), -1, (255,255,255), -1, 8)
            mask_im = np.array(mask_im, dtype=np.uint8)
            mask_im = cv2.cvtColor( mask_im, cv2.COLOR_BGR2GRAY)
            img_save_name = dir_path + 'ROI__' + str(count) + '__' + pathsplt[-1][0:-ffl] + '__layer__' + str(lyr) + '__' + str(ox) + '__' + str(oy) + '__' + str(w) + '__' + str(h)
            if(ff in openslide_formats):
                im_object = img.read_region((start_x,start_y),level,(w + 2*padd, h + 2*padd))
                im_object.save(img_save_name + '__.png')
                w1t,h1t = im_object.size 
                # print(im_object.shape)
                # raise valueError()
            else:
                im_patch = img[start_y:start_y + h + 2*padd, start_x:start_x + w + 2*padd, :]
                cv2.imwrite(img_save_name + '__.png', im_patch)
                h1t, w1t, ch1t = im_patch.shape
            mask_im = mask_im[0:h1t, 0:w1t]
            cv2.imwrite(img_save_name + '__mask__.png', mask_im)
            count = count + 1
        return count


list_xml =  glob.glob("/home/ravi/Desktop/TCGA-BRCA_DA/*.xml")
for item_xml in list_xml:
    print(item_xml)
    image_path = item_xml.replace(".xml",".svs")
    print(image_path)
    # img_name=image_path.split('/')[-1].split('.')[0]
    # print(img_name)
    c = get_annotations_1('.svs',item_xml,'/home/ravi/Desktop/TCGA-BRCA_DA/extract/',0,image_path,level=0)
    print("number of ROI: ",c)

/home/ravi/Desktop/TCGA-BRCA_DA/TCGA-A1-A0SF.xml
/home/ravi/Desktop/TCGA-BRCA_DA/TCGA-A1-A0SF.svs
ox and oy:  3338 6310
W and H:  3338 6310
ox and oy:  4295 5408
W and H:  4295 5408
ox and oy:  6289 4121
W and H:  6289 4121
ox and oy:  4328 3613
W and H:  4328 3613
ox and oy:  2706 5916
W and H:  2706 5916
ox and oy:  3058 2825
W and H:  3058 2825
number of ROI:  6
/home/ravi/Desktop/TCGA-BRCA_DA/TCGA-3C-AALK.xml
/home/ravi/Desktop/TCGA-BRCA_DA/TCGA-3C-AALK.svs
ox and oy:  2958 1387
W and H:  2958 1387
ox and oy:  7095 2906
W and H:  7095 2906
ox and oy:  8225 7169
W and H:  8225 7169
ox and oy:  6389 4863
W and H:  6389 4863
ox and oy:  5787 1659
W and H:  5787 1659
number of ROI:  5


In [3]:
print(c)

10


In [4]:
'''
# See export_annotated_region_lib.Window.py for more details about how this class works
class ExportAnnotatedRegionWindow(Window):
    __gtype_name__ = "ExportAnnotatedRegionWindow"
    
    def finish_initializing(self, builder): # pylint: disable=E1002
        """Set up the main window"""
        super(ExportAnnotatedRegionWindow, self).finish_initializing(builder)

        self.AboutDialog = AboutExportAnnotatedRegionDialog
        self.PreferencesDialog = PreferencesExportAnnotatedRegionDialog

        # Code for other initialization actions should be added here 

        self.button_count = 0                     # Allows only one execution per click
        userhome = os.path.expanduser('~')
        self.directory = userhome + '/Desktop/Extracted_Regions'       # Default directory to the images
        # if not os.path.exists(self.directory):
        #     os.mkdir(self.directory)
        self.directory_control = 0    #  Controls which one to run launch_hardinfo_0  or  launch_hardinfo_1
        self.format = '.ndpi'         #  User selects the image format using drop down menu
        self.padd = 112               #  The padding used
        self.xml_toggle = True        # Check button


    def on_sourcefolder_file_set(self, widget):
        self.root_path = widget.get_filename()
        print("Source Folder Choosen: ", self.root_path)
        return self.root_path

    def on_xmlfolder_file_set(self, widget):
        self.xml_path = widget.get_filename()
        print("XML Folder Choosen: ", self.xml_path)
        return self.xml_path

    def on_storefolder_file_set(self, widget):
        self.directory = widget.get_filename()
        print("Store Folder Choosen: ", self.directory)
        return self.directory


    def on_checkbutton1_toggled(self, widget, data=None):
        if(self.xml_toggle):
            self.xml_path = self.root_path
            self.ui.xmlfolder.set_sensitive(False)
            self.directory_control = 1
            self.xml_toggle = False
        else:
            self.xml_path = ''
            self.ui.xmlfolder.set_sensitive(True)
            self.directory_control = 0
            self.xml_toggle = True

    def on_combobox1_changed(self, combo, data=None):
        tree_iter = combo.get_active_iter()
        if tree_iter is not None:
            model = combo.get_model()
            self.format = model[tree_iter][0]
            print("Selected: Image Format=%s" % self.format)
            # print(len(self.format))


    def on_entry1_activate(self,widget, data=None):
        self.padd = int(widget.get_text())
        print("Padding :", self.padd)


    def launch_hardinfo_0(self):
        ffl = len(self.format)
        root_path = self.root_path + '/'
        xml_path = self.xml_path + '/'
        directory = self.directory + '/'
        if not os.path.exists(directory):
            os.mkdir(directory)
        sub_fold = os.listdir(xml_path)
        print(sub_fold)
        n_fold = len(sub_fold)
        lt1 = root_path.split('/')
        lp1 = len(lt1) - 1
        for i_1 in range(n_fold):
            gdouble_fraction = float(i_1)/n_fold
            self.ui.progressbar1.set_fraction(gdouble_fraction)
            xml_file_name = xml_path + sub_fold[i_1]
            wsi = sub_fold[i_1][0:-4]
            print(wsi)
            for root,dirs,files in os.walk(root_path):
                for fl in files:
                    if(fl[0:-ffl] == wsi):
                        wsi_path = root + '/' + fl
            c = get_annotations_1(self, xml_file_name, wsi_path, lp1)
        gdouble_fraction = float(i_1)/n_fold
        self.ui.progressbar1.set_fraction(1.0)
        self.button_count = self.button_count - 1
        return


    def launch_hardinfo_1(self):
        root_path = self.root_path
        xml_path = self.xml_path + '/'
        directory = self.directory + '/'
        ffl = len(self.format)
        if not os.path.exists(directory):
            os.mkdir(directory)
        wsi_paths = []
        xml_files = []
        for root,dirs,files in os.walk(root_path):
            found = 0
            if(len(dirs) == 0):
                lf = len(files)
                if(lf != 0):
                    for fl1 in files:
                        for fl2 in files:
                            if(fl1[0:-4] == fl2[0:-ffl]):
                                if(fl1 != fl2 and (str(self.format) == str(fl2[-ffl:]))):
                                    wsi_paths.append(root + '/' + fl2)
                                    xml_files.append(root + '/' + fl1)
                                    break
                        #           found = 1
                        #           break
                        # if(found == 1):
                        #   break
        # print(wsi_paths)
        # print(xml_files)
        n_fold = len(xml_files)
        lt1 = root_path.split('/')
        lp1 = len(lt1)
        for i_1 in range(n_fold):
            gdouble_fraction = float(i_1)/n_fold
            self.ui.progressbar1.set_fraction(gdouble_fraction)
            c = get_annotations_1(self, xml_files[i_1], wsi_paths[i_1], lp1)
        self.ui.progressbar1.set_fraction(1.0)
        self.button_count = self.button_count - 1
        return


    def on_button1_clicked(self, widget, data=None):
        # print(self.button_count)
        if(self.button_count == 0):
            self.button_count = self.button_count + 1
            if(self.directory_control == 0):
                t1 = threading.Thread(target=self.launch_hardinfo_0)
                t1.daemon = True
                t1.start()
            else:
                t1 = threading.Thread(target=self.launch_hardinfo_1)
                t1.daemon = True
                t1.start()              

            # t1.join()
'''

'\n# See export_annotated_region_lib.Window.py for more details about how this class works\nclass ExportAnnotatedRegionWindow(Window):\n    __gtype_name__ = "ExportAnnotatedRegionWindow"\n    \n    def finish_initializing(self, builder): # pylint: disable=E1002\n        """Set up the main window"""\n        super(ExportAnnotatedRegionWindow, self).finish_initializing(builder)\n\n        self.AboutDialog = AboutExportAnnotatedRegionDialog\n        self.PreferencesDialog = PreferencesExportAnnotatedRegionDialog\n\n        # Code for other initialization actions should be added here \n\n        self.button_count = 0                     # Allows only one execution per click\n        userhome = os.path.expanduser(\'~\')\n        self.directory = userhome + \'/Desktop/Extracted_Regions\'       # Default directory to the images\n        # if not os.path.exists(self.directory):\n        #     os.mkdir(self.directory)\n        self.directory_control = 0    #  Controls which one to run launch_ha